In [10]:
import json

with open('documents-with-ids.json', 'r') as f_in:
    documents = json.load(f_in)

In [8]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks, such as:

["question1", "question2", ..., "question5"]
""".strip()

In [3]:
import anthropic

client = anthropic.Anthropic()

In [4]:
doc = documents[2]
prompt = prompt_template.format(**doc)

In [5]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.messages.create(
      model="claude-3-5-sonnet-20240620",
      max_tokens=1024,
      messages=[
        {"role": "user", "content": prompt}
      ]
    )

    json_response = response.content
    
    return json_response

In [6]:
from tqdm.auto import tqdm

In [7]:
results = {}

In [12]:
# when ran, it will take awhile and is an expensive process --- $4 to generate w/ openAI
# used the "results.bin" file instead

# for doc in tqdm(documents):
#   doc_id = doc['id']

#   if doc_id in results:
#     continue

#   questions = generate_questions(doc)
#   results[doc_id] = questions

In [13]:
import pickle

In [14]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [15]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [29]:
# the results are from openAI and not Anthropic

parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [31]:
# needed to parse so for...loop above works
# json_questions = [
#     r"How can I resolve the Docker error 'invalid mode: Program Files\Git\var\lib\postgresql\data'?",
#     "What should I do if I encounter an invalid mode error in Docker on Windows?",
#     "What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
#     "Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
#     r"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
# ]


In [32]:
# added due to parsing issue above. Can comment out or delete below and "json_questions"
# results[doc_id] = json.dumps(json_questions)

In [33]:
doc_index = {d['id']: d for d in documents}

In [35]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    
    for q in questions:
        final_results.append((q, course, doc_id))

In [36]:
final_results[:10]

[('When does the course begin?', 'data-engineering-zoomcamp', 'c02e79ef'),
 ('How can I get the course schedule?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('What is the link for course registration?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('How can I receive course announcements?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where do I join the Slack channel?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where can I find the prerequisites for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('How do I check the prerequisites for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Where are the course prerequisites listed?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('What are the requirements for joining this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Where is the list of prerequisites for the course?',
  'data-engineering-zoomcamp',
  '1f6520ca')]

In [37]:
import pandas as pd

In [38]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [39]:
# export as CSV

df.to_csv('ground-truth-data.csv', index=False)

In [40]:
!head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
